# COTS Augmentation Gallery using the Albumentations Library

* This notebook is a set of quick examples of augmentation techniques using the Albumentations Library

* It also briefly shows (at the end) how to move your bounding boxes with albumentations

For the full set of augmentations and their documentation, visit: https://albumentations.ai/docs/getting_started/transforms_and_targets/

## Utility Functions

In [ ]:
import albumentations as A

import warnings
warnings.filterwarnings("ignore")

import os
import torch
import importlib
import cv2 
import pandas as pd
import numpy as np

import ast
import shutil
import sys

from tqdm.notebook import tqdm
tqdm.pandas()

from PIL import Image
from IPython.display import display

In [ ]:
# Modified from https://www.kaggle.com/remekkinas/yolox-inference-on-kaggle-for-cots-lb-0-507
# Additions: 
#     allows customized box color (BGR)

def draw_yolox_predictions(img, bboxes, scores, bbclasses, classes_dict, boxcolor = (0,0,255)):
    outimg = img.copy()
    for i in range(len(bboxes)):
        box = bboxes[i]
        cls_id = int(bbclasses[i])
        score = scores[i]
        x0 = int(box[0])
        y0 = int(box[1])
        x1 = x0 + int(box[2])
        y1 = y0 + int(box[3])

        cv2.rectangle(outimg, (x0, y0), (x1, y1), boxcolor, 2)
        cv2.putText(outimg, '{}:{:.1f}%'.format(classes_dict[cls_id], score * 100), (x0, y0 - 3), cv2.FONT_HERSHEY_PLAIN, 0.8, boxcolor, thickness = 1)
    return outimg

## Pick Your Example Image Here

In [ ]:
%cd /kaggle/working

from sklearn.model_selection import GroupKFold

def get_bbox(annots):
    bboxes = [list(annot.values()) for annot in annots]
    return bboxes

def get_path(row):
    row['image_path'] = f'{ROOT_DIR}/train_images/video_{row.video_id}/{row.video_frame}.jpg'
    return row

ROOT_DIR  = '/kaggle/input/tensorflow-great-barrier-reef/'

df = pd.read_csv("/kaggle/input/tensorflow-great-barrier-reef/train.csv")

# Don't filter for annotated frames. Include frames with no bboxes as well!
df["num_bbox"] = df['annotations'].apply(lambda x: str.count(x, 'x'))
df_train = df

# Annotations 
df_train['annotations'] = df_train['annotations'].progress_apply(lambda x: ast.literal_eval(x))
df_train['bboxes'] = df_train.annotations.progress_apply(get_bbox)

df_train = df_train.progress_apply(get_path, axis=1)

kf = GroupKFold(n_splits = 5) 
df_train = df_train.reset_index(drop=True)
df_train['fold'] = -1
for fold, (train_idx, val_idx) in enumerate(kf.split(df_train, y = df_train.video_id.tolist(), groups=df_train.sequence)):
    df_train.loc[val_idx, 'fold'] = fold

df_test = df_train[df_train.fold == 4]

In [ ]:
image_paths = df_test.image_path.tolist()
gt = df_test.bboxes.tolist()

This is my favourite image. Because all the COTS are in the top left hand corner

In [ ]:
i = 1380

image_path = image_paths[i]
Img = Image.open(image_path)
display(Img)

# A cropped image for simplicity, and with boxes showing you where the COTS are

In [ ]:
img_np = np.array(Img)[:360,:640]
out_image = draw_yolox_predictions(img_np, gt[i], [1.0] * len(gt[i]), [0] * len(gt[i]), ['COTS'], (0,255,0))
display(Image.fromarray(out_image))

# The Albumentations

In [ ]:
def show_augmentation(img, augmentation):
    """
        img: a numpy array of the image
        augmentation: a function from the Albumentations library
        see https://albumentations.ai/docs/getting_started/transforms_and_targets/
        
        returns: a numpy array of the augmented image
    """
    transform = A.Compose([augmentation])
    img_aug = transform(image=img)['image']
    return(img_aug)

## Blurring Augmentations

### Blur

In [ ]:
AUGMENTATION = A.Blur(p = 1.0)

for q in range(1):
    img_aug = show_augmentation(img_np, AUGMENTATION)
    out_image = draw_yolox_predictions(img_aug, gt[i], [1.0] * len(gt[i]), [0] * len(gt[i]), ['COTS'], (0,255,0))
    display(Image.fromarray(out_image))

### GaussianBlur

In [ ]:
AUGMENTATION = A.GaussianBlur(p = 1.0)

for q in range(1):
    img_aug = show_augmentation(img_np, AUGMENTATION)
    out_image = draw_yolox_predictions(img_aug, gt[i], [1.0] * len(gt[i]), [0] * len(gt[i]), ['COTS'], (0,255,0))
    display(Image.fromarray(out_image))

### MedianBlur

In [ ]:
AUGMENTATION = A.MedianBlur(p = 1.0)

for q in range(1):
    img_aug = show_augmentation(img_np, AUGMENTATION)
    out_image = draw_yolox_predictions(img_aug, gt[i], [1.0] * len(gt[i]), [0] * len(gt[i]), ['COTS'], (0,255,0))
    display(Image.fromarray(out_image))

### Downscale

In [ ]:
AUGMENTATION = A.Downscale(scale_min=0.5, scale_max=0.5, p = 1.0)

for q in range(1):
    img_aug = show_augmentation(img_np, AUGMENTATION)
    out_image = draw_yolox_predictions(img_aug, gt[i], [1.0] * len(gt[i]), [0] * len(gt[i]), ['COTS'], (0,255,0))
    display(Image.fromarray(out_image))

### ImageCompression

In [ ]:
AUGMENTATION = A.ImageCompression(quality_lower=20, quality_upper=40, p = 1.0)

for q in range(1):
    img_aug = show_augmentation(img_np, AUGMENTATION)
    out_image = draw_yolox_predictions(img_aug, gt[i], [1.0] * len(gt[i]), [0] * len(gt[i]), ['COTS'], (0,255,0))
    display(Image.fromarray(out_image))

## Clarity Augmentations

### Sharpen

In [ ]:
AUGMENTATION = A.Sharpen(p = 1.0)

for q in range(1):
    img_aug = show_augmentation(img_np, AUGMENTATION)
    out_image = draw_yolox_predictions(img_aug, gt[i], [1.0] * len(gt[i]), [0] * len(gt[i]), ['COTS'], (0,255,0))
    display(Image.fromarray(out_image))

### CLAHE

In [ ]:
AUGMENTATION = A.CLAHE(p = 1.0)

for q in range(1):
    img_aug = show_augmentation(img_np, AUGMENTATION)
    out_image = draw_yolox_predictions(img_aug, gt[i], [1.0] * len(gt[i]), [0] * len(gt[i]), ['COTS'], (0,255,0))
    display(Image.fromarray(out_image))

### IAAAdditiveGaussianNoise

In [ ]:
AUGMENTATION = A.IAAAdditiveGaussianNoise(p = 1.0)

for q in range(1):
    img_aug = show_augmentation(img_np, AUGMENTATION)
    out_image = draw_yolox_predictions(img_aug, gt[i], [1.0] * len(gt[i]), [0] * len(gt[i]), ['COTS'], (0,255,0))
    display(Image.fromarray(out_image))

### Emboss

In [ ]:
AUGMENTATION = A.Emboss(p = 1.0)

for q in range(1):
    img_aug = show_augmentation(img_np, AUGMENTATION)
    out_image = draw_yolox_predictions(img_aug, gt[i], [1.0] * len(gt[i]), [0] * len(gt[i]), ['COTS'], (0,255,0))
    display(Image.fromarray(out_image))

## Color Augmentations

### HSV Augmentation

In [ ]:
AUGMENTATION = A.HueSaturationValue(p = 1.0)

for q in range(1):
    img_aug = show_augmentation(img_np, AUGMENTATION)
    out_image = draw_yolox_predictions(img_aug, gt[i], [1.0] * len(gt[i]), [0] * len(gt[i]), ['COTS'], (0,255,0))
    display(Image.fromarray(out_image))

### RGBShift

In [ ]:
AUGMENTATION = A.RGBShift(p = 1.0)

for q in range(1):
    img_aug = show_augmentation(img_np, AUGMENTATION)
    out_image = draw_yolox_predictions(img_aug, gt[i], [1.0] * len(gt[i]), [0] * len(gt[i]), ['COTS'], (0,255,0))
    display(Image.fromarray(out_image))

### RandomGamma

In [ ]:
AUGMENTATION = A.RandomGamma(p = 1.0)

for q in range(1):
    img_aug = show_augmentation(img_np, AUGMENTATION)
    out_image = draw_yolox_predictions(img_aug, gt[i], [1.0] * len(gt[i]), [0] * len(gt[i]), ['COTS'], (0,255,0))
    display(Image.fromarray(out_image))

### RandomContrast

In [ ]:
AUGMENTATION = A.RandomContrast(p = 1.0)

for q in range(1):
    img_aug = show_augmentation(img_np, AUGMENTATION)
    out_image = draw_yolox_predictions(img_aug, gt[i], [1.0] * len(gt[i]), [0] * len(gt[i]), ['COTS'], (0,255,0))
    display(Image.fromarray(out_image))

### RandomBrightness

In [ ]:
AUGMENTATION = A.RandomBrightness(p = 1.0)

for q in range(1):
    img_aug = show_augmentation(img_np, AUGMENTATION)
    out_image = draw_yolox_predictions(img_aug, gt[i], [1.0] * len(gt[i]), [0] * len(gt[i]), ['COTS'], (0,255,0))
    display(Image.fromarray(out_image))

### ChannelShuffle

In [ ]:
AUGMENTATION = A.ChannelShuffle(p = 1.0)

for q in range(1):
    img_aug = show_augmentation(img_np, AUGMENTATION)
    out_image = draw_yolox_predictions(img_aug, gt[i], [1.0] * len(gt[i]), [0] * len(gt[i]), ['COTS'], (0,255,0))
    display(Image.fromarray(out_image))

# Compound Augmentations

* Includes support for spatial augmentations (i.e. those that alter object location, therefore the need to move the bboxes)

In [ ]:
def show_compound_augmentation(img, bboxes, labels, augmentation_list):
    """
        img: a numpy array of the image
        bboxes: COCO-format bounding boxes
        labels: a list of labels
        augmentation_list: a list of functions from the Albumentations library
        see https://albumentations.ai/docs/getting_started/transforms_and_targets/
        
        returns: a numpy array of the augmented image
    """
    transform = A.Compose(augmentation_list,
        bbox_params=A.BboxParams(
        format='coco',
        label_fields=['class_labels']
    ))
    transformed = transform(image=img, bboxes=bboxes, class_labels = labels)
    img_aug = transformed['image']
    boxes = np.array([list(b) for b in transformed['bboxes']])
    labels = np.array(transformed['class_labels'])
    return img_aug, boxes, labels

* One can include a conga list of their favourite augmentations
* This example also shows how to move bounding boxes when the augmentation moves the image

In [ ]:
# Example taken from https://analyticsindiamag.com/hands-on-guide-to-albumentation/

AUGMENTATION_LIST = [
        A.RandomRotate90(),
        A.Flip(),
        A.Transpose(),
        A.OneOf([
            A.MotionBlur(p=.2),
            A.MedianBlur(blur_limit=3, p=0.3),
            A.Blur(blur_limit=3, p=0.1),
        ], p=0.2),
        A.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.2, rotate_limit=45, p=0.2),
        A.OneOf([
            A.CLAHE(clip_limit=2),
            A.RandomBrightnessContrast(),            
        ], p=0.3),
        A.HueSaturationValue(p=0.3),
    ]

for q in range(5):
    img_aug, bboxes, labels = show_compound_augmentation(img_np, gt[i], [0] * len(gt[i]), AUGMENTATION_LIST)
    out_image = draw_yolox_predictions(img_aug, bboxes, [1.0] * len(bboxes), labels, ['COTS'], (0,255,0))
    display(Image.fromarray(out_image))

# Affine Transformation with BBoxes

In [ ]:
AUGMENTATION_LIST = [A.Affine(scale = (0.8, 1.2), translate_percent = 0.1, rotate = (-45,45), shear = (-5, 5), cval = (114,114,114), p = 1.0)]

for q in range(5):
    img_aug, bboxes, labels = show_compound_augmentation(img_np, gt[i], [0] * len(gt[i]), AUGMENTATION_LIST)
    out_image = draw_yolox_predictions(img_aug, bboxes, [1.0] * len(bboxes), labels, ['COTS'], (0,255,0))
    display(Image.fromarray(out_image))